In [1]:
import numpy as np
import skimage.measure
import pydicom
import vtk
from vtk.util import numpy_support
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
import gc
gc.collect()

In [2]:
plt.gray()
os.chdir('C:/Users/span/Documents/3DSlicerTutorial/CNN.test/')
number1 = "23"

<Figure size 432x288 with 0 Axes>

In [3]:
PathDicom = "./"+str(number1)+"_dl_input/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()

# Load dimensions using `GetDataExtent`
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()

# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)

# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')

a_in = ArrayDicom.copy()

In [4]:
PathDicom = "./"+str(number1)+"_dl_mask/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()

# Load dimensions using `GetDataExtent`
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()

# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)

# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')

a_mask = ArrayDicom.copy()

In [5]:
assert a_in.shape == a_mask.shape

In [6]:
import random
random.seed(a=1024, version=2)
list1 = list(range(a_in.shape[2]))
trainlist1 = random.sample(list1, k=round(a_in.shape[2]*0.6))
testlist1 = [x for x in list1 if (x not in trainlist1)]

In [ ]:
image = a_in[:, :, i].astype(np.uint8).copy()
image1 = rescale(image, (512/a_in[:, :, i].shape[1]), anti_aliasing=False)
image2 = (((image1 - image1.min()) / (image1.max() - image1.min())) * 255.9).astype(np.uint8)
img0 = Image.fromarray(image2.astype(np.uint8), mode = "L")
plt.imshow(img0)
image3 = image2[..., np.newaxis]
# img0.save('./data/train_image/23_train_image/im0.jpg')
plt.imsave('./data/train_image/23_train_image/im0.png', image2, format='png', cmap='gray')
# cv2.imwrite('./data/train_image/23_train_image/img-png-gray.jpg', image3[:, :,0].astype(np.uint8)) 

In [ ]:
img22 = cv2.imread('./data/train_image/23_train_image/im0.png')
img22.shape
# img23 = cv2.imread('./data/train_image/23_train_image/img-png-gray.jpg')
# print(img23.shape,img22.shape)

In [9]:
from PIL import Image
from skimage.transform import rescale
# output  23_train_image
path1 = "data/train_image/23_train_image/train/"
path2 = "data/train_mask/23_train_mask/train/"
path3 = "data/val_image/23_test_image/val/"
path4 = "data/val_mask/23_test_mask/val/"

resize1 = 256

# output  23_train_image
for i in trainlist1:
    image = a_in[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path1+str(number1)+"_taimage_"+str("{0:03}".format(i))+".png")

<ipython-input-9-e8bcdc10030f>:15: RuntimeWarning: invalid value encountered in true_divide
  image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)


In [10]:
# output  23_train_mask

for i in trainlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path2+str(number1)+"_tamask_"+str("{0:03}".format(i))+".png")

# output  23_train_image

for i in testlist1:
    image = a_in[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path3+str(number1)+"_tsimage_"+str("{0:03}".format(i))+".png")
# output  23_train_mask

for i in testlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path4+str(number1)+"_tsmask_"+str("{0:03}".format(i))+".png")

<ipython-input-10-76bb4e6b7e79>:6: RuntimeWarning: invalid value encountered in true_divide
  image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
<ipython-input-10-76bb4e6b7e79>:15: RuntimeWarning: invalid value encountered in true_divide
  image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
<ipython-input-10-76bb4e6b7e79>:23: RuntimeWarning: invalid value encountered in true_divide
  image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)


In [ ]:
i1 = 25
i2 = 45
fig, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(16, 6.4),
                                    sharex=True, sharey=True)
ax0.imshow(a_in[:, :, trainlist1[i1]], cmap='gray')
ax0.axis('off')
ax0.set_title('a in 1')
ax1.imshow(a_mask[:, :, trainlist1[i1]], cmap='gray')
ax1.axis('off')
ax1.set_title('a out 1')
ax2.imshow(a_in[:, :, trainlist1[i2]], cmap='gray') # 'magma'
ax2.axis('off')
ax2.set_title('a in 2')
ax3.imshow(a_mask[:, :, trainlist1[i2]], cmap='gray')
ax3.axis('off')
ax3.set_title('a out 2')

fig.tight_layout()
plt.show()

GENERATORS

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True
)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
path1 = "data/train_image/23_train_image"
path2 = "data/train_mask/23_train_mask"
path3 = "data/val_image/23_test_image"
path4 = "data/val_mask/23_test_mask"

train_image_generator = train_datagen.flow_from_directory(
    path1,
    color_mode = "grayscale",
    batch_size = 4 #NORMALLY 4/8/16/32
)

train_mask_generator = train_datagen.flow_from_directory(
    path2,
    color_mode = "grayscale",
    batch_size = 4 #NORMALLY 4/8/16/32
)

val_image_generator = val_datagen.flow_from_directory(
    path3,
    color_mode = "grayscale",
    batch_size = 4 #NORMALLY 4/8/16/32
)


val_mask_generator = val_datagen.flow_from_directory(
    path4,
    color_mode = "grayscale",
    batch_size = 4 #NORMALLY 4/8/16/32
)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 337 images belonging to 1 classes.
Found 337 images belonging to 1 classes.
Found 225 images belonging to 1 classes.
Found 225 images belonging to 1 classes.


In [13]:
import keras
import tensorflow as tf

def get_model(optimizer, loss_metric, metrics, lr=1e-3):
    inputs = keras.Input((sample_width, sample_height, 1))
    conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(inputs)
    conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(conv1)
    conv1 = tf.reshape(conv1, tf.shape(conv1))
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    drop1 = keras.layers.Dropout(0.5)(pool1)

    conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(drop1)
    conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(conv2)
    conv2 = tf.reshape(conv2, tf.shape(conv2))
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    drop2 = keras.layers.Dropout(0.5)(pool2)

    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(drop2)
    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(conv3)
    conv3 = tf.reshape(conv3, tf.shape(conv3))
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    drop3 = keras.layers.Dropout(0.3)(pool3)

    conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(drop3)
    conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(conv4)
    conv4 = tf.reshape(conv4, tf.shape(conv4))
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    drop4 = keras.layers.Dropout(0.3)(pool4)

    conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(drop4)
    conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(conv5)
    conv5 = tf.reshape(conv5, tf.shape(conv5))
    
    conv5u = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv5)
    conv5u = tf.reshape(conv5u, tf.shape(conv5u))
    up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
    conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(up6)
    conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(conv6)
    conv6 = tf.reshape(conv6, tf.shape(conv6))
    
    conv6u = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv6)
    conv6u = tf.reshape(conv6u, tf.shape(conv6u))
    up7 = keras.layers.concatenate([conv6u, conv3], axis=3)
    conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(up7)
    conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(conv7)
    conv7 = tf.reshape(conv7, tf.shape(conv7))
    
    conv7u = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv7)
    conv7u = tf.reshape(conv7u, tf.shape(conv7u))
    up8 = keras.layers.concatenate([conv7u, conv2], axis=3)
    conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(up8)
    conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(conv8)
    conv8 = tf.reshape(conv8, tf.shape(conv8))
    
    conv8u = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv8)
    conv8u = tf.reshape(conv8u, tf.shape(conv8u))
    up9 = keras.layers.concatenate([conv8u, conv1], axis=3)
    conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(up9)
    conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(conv9)
    conv9 = tf.reshape(conv9, tf.shape(conv9))

    conv10 = keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    conv10 = tf.reshape(conv10, tf.shape(conv10))

    model = keras.Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=optimizer(lr=lr), loss=loss_metric, metrics=metrics)
    return model

In [14]:
smooth = 1.
# Dice Coefficient to work with Tensorflow
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Dice Coefficient to work outside Tensorflow
def dice_coef_2(y_true, y_pred):
    side = len(y_true[0])
    y_true_f = y_true.reshape(side*side)
    y_pred_f = y_pred.reshape(side*side)
    intersection = sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (sum(y_true_f) + sum(y_pred_f) + smooth)

In [15]:
sample_width = 256
sample_height = 256

model = get_model(optimizer=tf.keras.optimizers.Adam, loss_metric=dice_coef_loss, metrics=[dice_coef], lr=1e-3)

In [ ]:
# fit the keras model on the dataset
# historyKe = model.fit(train_image_generator, train_mask_generator, epochs=150, batch_size=10)

# train_generator = zip(train_image_generator, train_mask_generator)
# val_generator = zip(val_image_generator, val_mask_generator)
from keras import backend as K

historyTF = model.fit(train_generator, epochs=10, validation_data=val_generator)


Epoch 1/10
    453/Unknown - 1994s 4s/step - loss: -0.0098 - dice_coef: 0.0130

In [ ]:
model.save('Model_ep10.h5')

In [ ]:
from keras import backend as K

NO_OF_EPOCHS = 30
NO_OF_TRAINING_IMAGES = len(os.listdir('./data/train_image/23_train_image/'))
NO_OF_VAL_IMAGES = len(os.listdir('./data/val_image/23_test_image/'))
BATCH_SIZE = 4

# checkpoint = ModelCheckpoint(weights_path, monitor='METRIC_TO_MONITOR', 
#                              verbose=1, save_best_only=True, mode='max')
# csv_logger = CSVLogger('./log.out', append=True, separator=';')
# earlystopping = EarlyStopping(monitor = 'METRIC_TO_MONITOR', verbose = 1,
#                               min_delta = 0.01, patience = 3, mode = 'max')

# callbacks_list = [checkpoint, csv_logger, earlystopping]

results = model.fit_generator(train_generator, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_generator, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE))

CNN layer calculate 

In [ ]:
# [filter size, stride, padding]
#Assume the two dimensions are the same
#Each kernel requires the following parameters:
# - k_i: kernel size
# - s_i: stride
# - p_i: padding (if padding is uneven, right padding will higher than left padding; "SAME" option in tensorflow)
# 
#Each layer i requires the following parameters to be fully represented: 
# - n_i: number of feature (data layer has n_1 = imagesize )
# - j_i: distance (projected to image pixel distance) between center of two adjacent features
# - r_i: receptive field of a feature in layer i
# - start_i: position of the first feature's receptive field in layer i (idx start from 0, negative means the center fall into padding)

import math
convnet =   [[11,4,0],[3,2,0],[5,1,2],[3,2,0],[3,1,1],[3,1,1],[3,1,1],[3,2,0],[6,1,0], [1, 1, 0]]
layer_names = ['conv1','pool1','conv2','pool2','conv3','conv4','conv5','pool5','fc6-conv', 'fc7-conv']
imsize = 227

def outFromIn(conv, layerIn):
  n_in = layerIn[0]
  j_in = layerIn[1]
  r_in = layerIn[2]
  start_in = layerIn[3]
  k = conv[0]
  s = conv[1]
  p = conv[2]
  
  n_out = math.floor((n_in - k + 2*p)/s) + 1
  actualP = (n_out-1)*s - n_in + k 
  pR = math.ceil(actualP/2)
  pL = math.floor(actualP/2)
  
  j_out = j_in * s
  r_out = r_in + (k - 1)*j_in
  start_out = start_in + ((k-1)/2 - pL)*j_in
  return n_out, j_out, r_out, start_out
  
def printLayer(layer, layer_name):
  print(layer_name + ":")
  print("\t n features: %s \n \t jump: %s \n \t receptive size: %s \t start: %s " % (layer[0], layer[1], layer[2], layer[3]))
 
layerInfos = []
if __name__ == '__main__':
#first layer is the data layer (image) with n_0 = image size; j_0 = 1; r_0 = 1; and start_0 = 0.5
  print ("-------Net summary------")
  currentLayer = [imsize, 1, 1, 0.5]
  printLayer(currentLayer, "input image")
  for i in range(len(convnet)):
    currentLayer = outFromIn(convnet[i], currentLayer)
    layerInfos.append(currentLayer)
    printLayer(currentLayer, layer_names[i])
  print ("------------------------")
  layer_name = raw_input ("Layer name where the feature in: ")
  layer_idx = layer_names.index(layer_name)
  idx_x = int(raw_input ("index of the feature in x dimension (from 0)"))
  idx_y = int(raw_input ("index of the feature in y dimension (from 0)"))
  
  n = layerInfos[layer_idx][0]
  j = layerInfos[layer_idx][1]
  r = layerInfos[layer_idx][2]
  start = layerInfos[layer_idx][3]
  assert(idx_x < n)
  assert(idx_y < n)
  
  print ("receptive field: (%s, %s)" % (r, r))
  print ("center: (%s, %s)" % (start+idx_x*j, start+idx_y*j))